In [ ]:
# Importiamo le librerie

from itertools import combinations
import matplotlib.pyplot as plt
import pandas as pd
import pickle
from itertools import combinations
from os import listdir
from os.path import isfile, join

In [ ]:
# Variabili globali

mypath = 'dataset'

# Mi vado a prendere i path di tutti i file nella cartella dataset

onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

# Mi vado a generare tutte le possibili combinazioni di due elementi da un insieme di 5 elementi


comb = list(combinations(range(0,5), 2))

In [ ]:
# Andiamo a leggere i file pickle per importare i dataset

all = []

all_document = []

document = 'Friday-02-03-2018_TrafficForML_CICFlowMeter'

for a in onlyfiles:
    b = a.replace('.csv','')
    with open('pickle/'+ document +'/scaled_document.pickle', 'rb') as handle:
        all.append(pickle.load(handle))

    with open('pickle/'+'Friday-02-03-2018_TrafficForML_CICFlowMeter'+'/scaled_document_plot.pickle', 'rb') as handle:
        all_document.append(pickle.load(handle))

for i, j in enumerate(onlyfiles):
    if j == document:
        idx = i

folder = onlyfiles[idx]
folder = folder.replace('.csv','')
print(folder)

In [ ]:
# Andiamo a pulire tutti i dataset

# Come prima cosa andiamo a leggere le feature dai vari file csv

'''for a in onlyfiles:
    '''
anova = pd.read_csv('top_feature/'+'Friday-23-02-2018_TrafficForML_CICFlowMeter'+"/anova.csv")
tree = pd.read_csv('top_feature/'+'Friday-23-02-2018_TrafficForML_CICFlowMeter'+"/ExtraTree.csv")
svm = pd.read_csv('top_feature/'+'Friday-23-02-2018_TrafficForML_CICFlowMeter'+"/SVM.csv")
rfe = pd.read_csv('top_feature/'+'Friday-23-02-2018_TrafficForML_CICFlowMeter'+"/RFE.csv")


In [ ]:
display(svm)

In [ ]:
# Facciamo una concatenazione dei vari dataframe.
# Saranno utilizzati per prendere le feature che si ripetono di più e che hanno i punteggi maggiori

total_feature = pd.concat([anova,tree,svm,rfe],ignore_index=True)
total_feature_count = pd.concat([anova,tree,svm,rfe],ignore_index=True)

In [ ]:
# Andiamo a eseguire un operazione di group by, con successivo count delle occorrenze dei vari campi

total_feature_count = total_feature_count.groupby(['Input_Features']).count().reset_index().sort_values(by='F_Score',ascending=False)

# Andiamo a rinominare la colonna F_Score, che adesso contiene i valori di count

total_feature_count.rename(columns = {'F_Score':'Conto'}, inplace = True)
print(total_feature_count)

In [ ]:
# Delle delle feature più frequenti mi interessano le prime dieci

most_frequent = total_feature_count[:10]

print(most_frequent)

In [ ]:
# Ci serve ancora il parametro F_Scores, quindi creiamo un nuovo dataframe con queste righe.
# Abbiamo le dieci righe, però, sono ancora presenti le ridondanze che prima avevamo eliminato.

test = pd.DataFrame(data=[],columns=total_feature.columns)

for a in most_frequent['Input_Features']:
    test = pd.concat([test,total_feature[total_feature['Input_Features'] == a]])

print(test)

In [ ]:
# Creo una variabile all'interno del quale vado a salvare il groupby del precedente dataframe, solo che in uesto caso
# andiamo a sommare i punteggi di F_Scores, invece di contare le occorrenze.

group_by = test.groupby(['Input_Features'])['F_Score'].sum().reset_index()

In [ ]:
print(group_by)

In [ ]:
# Facciamo un ordinamento del dataframe precedente

print(group_by.sort_values(by='F_Score',ascending=False))

In [ ]:
# A questo punto, terminiamo l'operazione andando a moltiplicare la frequenza per la somma dei punteggi delle varie feature.

last_feature = pd.DataFrame(data=[],columns=group_by.columns)

for a,b in zip(most_frequent['Input_Features'],most_frequent['Conto']):
    last_feature.loc[len(last_feature)] =  {'Input_Features':a,'F_Score':group_by[group_by['Input_Features'] == a]['F_Score'].values[0]*b}
    

In [ ]:
# Facciamo una stampa ordinata

print(last_feature.sort_values(by='F_Score',ascending=False))

In [ ]:
# A questo punto ci interessano solo le prime 5 feature, e tutte le loro possibili combinazioni di due elementi.

esito_finale = pd.DataFrame(data=[],columns=['Feature_1','Feature_2'])
top_index = []

# Qui andiamo a prendere gli indici delle prime 5 features

for a in range(0,5):
    top_index.append(last_feature.sort_values(by='F_Score',ascending=False).index[a])
    
# Andiamo a inserire tutte le possibili combinazioni nel dataframe

for b in comb:
    print(last_feature.loc[top_index[b[0]]]['Input_Features'])
    print(last_feature.loc[top_index[b[1]]]['Input_Features'])
    esito_finale.loc[len(esito_finale)] =  {
        'Feature_1': last_feature.loc[top_index[b[0]]]['Input_Features'],
        'Feature_2': last_feature.loc[top_index[b[1]]]['Input_Features']
        }

In [ ]:
print(esito_finale)

In [ ]:
# Salviamo l'esito finale in un file csv

esito_finale.to_csv('top_feature/'+folder+'/combinazioni.csv',index=False)